# L1: Language modelling

In this lab you will implement and train two neural language models: the fixed-window model and the recurrent neural network model. You will evaluate these models by computing their perplexity on a benchmark dataset.

In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch

For this lab, you should use the GPU if you have one:

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Data

The data for this lab is [WikiText](https://blog.einstein.ai/the-wikitext-long-term-dependency-language-modeling-dataset/), a collection of more than 100 million tokens extracted from the “Good” and “Featured” articles on Wikipedia. We will use the small version of the dataset, which contains slightly more than 2.5 million tokens.

The next cell contains code for an object that will act as a container for the “training” and the “validation” section of the data. We fill this container by reading the corresponding text files. The only processing we do is to whitespace-tokenise, and to enclose each non-empty line within `<bos>` (beginning-of-sentence) and `<eos>` (end-of-sentence) tokens.

In [5]:
class WikiText(object):

    def __init__(self):
        self.vocab = {}
        self.train = self.read_data('/content/drive/MyDrive/tdde09/wiki.train.tokens')
        self.valid = self.read_data('/content/drive/MyDrive/tdde09/wiki.valid.tokens')

    def read_data(self, path):
        ids = []
        with open(path, encoding='utf-8') as source:
            for line in source:
                line = line.rstrip()
                if line:
                    for token in ['<bos>'] + line.split() + ['<eos>']:
                        if token not in self.vocab:
                            self.vocab[token] = len(self.vocab)
                        ids.append(self.vocab[token])
        return ids

The cell below loads the data and prints the total number of tokens and the size of the vocabulary.

In [6]:
wikitext = WikiText()

print('Tokens in train:', len(wikitext.train))
print('Tokens in valid:', len(wikitext.valid))
print('Vocabulary size:', len(wikitext.vocab))
print(wikitext.vocab)

Tokens in train: 2099444
Tokens in valid: 218808
Vocabulary size: 33279
{'<bos>': 0, '=': 1, 'Valkyria': 2, 'Chronicles': 3, 'III': 4, '<eos>': 5, 'Senjō': 6, 'no': 7, '3': 8, ':': 9, '<unk>': 10, '(': 11, 'Japanese': 12, '戦場のヴァルキュリア3': 13, ',': 14, 'lit': 15, '.': 16, 'of': 17, 'the': 18, 'Battlefield': 19, ')': 20, 'commonly': 21, 'referred': 22, 'to': 23, 'as': 24, 'outside': 25, 'Japan': 26, 'is': 27, 'a': 28, 'tactical': 29, 'role': 30, '@-@': 31, 'playing': 32, 'video': 33, 'game': 34, 'developed': 35, 'by': 36, 'Sega': 37, 'and': 38, 'Media.Vision': 39, 'for': 40, 'PlayStation': 41, 'Portable': 42, 'Released': 43, 'in': 44, 'January': 45, '2011': 46, 'it': 47, 'third': 48, 'series': 49, 'same': 50, 'fusion': 51, 'real': 52, 'time': 53, 'gameplay': 54, 'its': 55, 'predecessors': 56, 'story': 57, 'runs': 58, 'parallel': 59, 'first': 60, 'follows': 61, '"': 62, 'Nameless': 63, 'penal': 64, 'military': 65, 'unit': 66, 'serving': 67, 'nation': 68, 'Gallia': 69, 'during': 70, 'Second'

## Problem 1: Fixed-window model

In this section, you will implement and train the fixed-window neural language model proposed by [Bengio et al. (2003)](http://www.jmlr.org/papers/volume3/bengio03a/bengio03a.pdf) and presented in the lectures. Recall that an input to the network takes the form of a vector of $n-1$ integers representing the preceding words. Each integer is mapped to a vector via an embedding layer. (All positions share the same embedding.) The embedding vectors are then concatenated and sent through a two-layer feed-forward network with a non-linearity in the form of a rectified linear unit (ReLU) and a final softmax layer.

### Problem 1.1: Vectorise the data (1&nbsp;point)

Your first task is to write code for transforming the data in the WikiText container into a vectorised form that can be fed to the fixed-window model. Concretely, you will implement a [collate function](https://pytorch.org/docs/stable/data.html#dataloader-collate-fn) in the form of a callable vectoriser object. Complete the skeleton code in the cell below:

In [7]:
class FixedWindowVectorizer(object):
    def __init__(self, n):
        # n-gram order
        self.n = n

    def __call__(self, data):
        all_ngrams = []

        for i in range(len(data) - self.n + 1):
          all_ngrams.append((data[i:i+self.n]))

        xs = []
        ys = []

        for ngram in all_ngrams:
            xs.append(ngram[:-1])
            ys.append(ngram[-1])


        # Convert inputs and outputs into tensors
        tensor_x = torch.LongTensor(xs)
        tensor_y = torch.LongTensor(ys)

        return tensor_x, tensor_y

Your code should implement the following specification:

**__init__** (*self*, *n*)

> Creates a new vectoriser with n-gram order $n$. Your code should be able to handle arbitrary n-gram orders $n \geq 1$.

**__call__** (*self*, *data*)

> Transforms WikiText *data* (a list of word ids) into a pair of tensors $\mathbf{X}$, $\mathbf{y}$ that can be used to train the fixed-window model. Let $N$ be the total number of $n$-grams from the token list; then $\mathbf{X}$ is a matrix with shape $(N, n-1)$ and $\mathbf{y}$ is a vector with length $N$.

#### 🤞 Test your code

Test your implementation by running the code in the next cell. Does the output match your expectation?

In [8]:
valid_x, valid_y = FixedWindowVectorizer(3)(wikitext.valid)

print(valid_x.size(), valid_y.size())

torch.Size([218806, 2]) torch.Size([218806])


### Problem 1.2: Implement the model (2&nbsp;points)

Your next task is to implement the fixed-window model based on the graphical specification given in the lecture.

In [9]:
import torch.nn as nn

class FixedWindowModel(nn.Module):

    def __init__(self, n, n_words, embedding_dim=50, hidden_dim=50):
        super().__init__()
        self.e = nn.Embedding(n_words, embedding_dim)
        self.linear1 = nn.Linear(embedding_dim*(n-1), hidden_dim)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(hidden_dim, n_words)

    def forward(self, x):
        embedded_x = self.e(x)
        concatenated_x = embedded_x.view(embedded_x.shape[0], -1)
        linear1_output = self.linear1(concatenated_x)
        relu_output = self.relu(linear1_output)
        return self.linear2(relu_output)

In [10]:
batch_size = 1
word_size = 10
n = 3
model = FixedWindowModel(n, word_size)
batch = torch.randint(0 , word_size, (batch_size, n-1))
output = model(batch)
output.shape

torch.Size([1, 10])

Here is the specification of the two methods:

**__init__** (*self*, *n*, *n_words*, *embedding_dim*=50, *hidden_dim*=50)

> Creates a new fixed-window neural language model. The argument *n* specifies the model&rsquo;s $n$-gram order. The argument *n_words* is the number of words in the vocabulary. The arguments *embedding_dim* and *hidden_dim* specify the dimensionalities of the embedding layer and the hidden layer of the feedforward network, respectively; their default value is 50.

**forward** (*self*, *x*)

> Computes the network output on an input batch *x*. The shape of *x* is $(B, n-1)$, where $B$ is the batch size. The output of the forward pass is a tensor of shape $(B, V)$ where $V$ is the number of words in the vocabulary.

**Hint:** The most efficient way to implement the vector concatenation in this model is to use the [`view()`](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.view) method.

#### 🤞 Test your code

Test your code by instantiating the model and feeding it a batch of examples from the training data.

### Problem 1.3: Train the model (3&nbsp;points)

Your final task is to write code to train the fixed-window model using minibatch gradient descent and the cross-entropy loss function. This should be a straightforward generalisation of the training loops that you have seen so far. Complete the skeleton code in the cell below:

In [11]:
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.nn.functional as F
import math

def train_fixed_window(n, n_epochs=2, batch_size=3200, lr=1e-2):


    # Initialise the model
    model = FixedWindowModel(n, len(wikitext.vocab)).to(device)

    # Initialise the optimiser (stochastic gradient descent)
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # Initialise the data loader
    data_loader = DataLoader(wikitext.train, batch_size=batch_size, collate_fn=FixedWindowVectorizer(n))
    valid_loader = DataLoader(wikitext.valid, batch_size=batch_size, collate_fn=FixedWindowVectorizer(n))


    # We train for the specified number of epochs
    for epoch in range(n_epochs):

        total_loss = 0

        model.train()

        # In each epoch, we loop over the batches provided by the data loader
        for batch_x, batch_y in data_loader:

            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)

            # Reset the accumulated gradients
            optimizer.zero_grad()

            # Forward pass
            output = model.forward(batch_x)

            # Compute the loss
            loss = F.cross_entropy(output, batch_y)

            # Backward pass: propagate the loss and compute the gradients
            loss.backward()

            # Update the parameters
            optimizer.step()


        running_loss = 0

        with torch.no_grad():

          model.eval()
          num_of_samples = 0

          # In each epoch, we loop over the batches provided by the data loader
          for batch_x, batch_y in valid_loader:

              batch_x = batch_x.to(device)
              batch_y = batch_y.to(device)

              # Forward pass
              output = model.forward(batch_x)

              sample_size = len(batch_x)

              # Compute the loss
              loss = F.cross_entropy(output, batch_y) * sample_size
              num_of_samples += sample_size


              running_loss += loss

        # Compute the average
        total_loss = (running_loss / num_of_samples)

        # Print as perplexity
        print(f'{math.exp(total_loss):.1f}')


    return model

Here is the specification of the training function:

**train_fixed_window** (*n*, *n_epochs* = 1, *batch_size* = 3200, *lr* = 0.01)

> Trains a fixed-window neural language model of order *n* using minibatch gradient descent and returns it. The parameters *n_epochs* and *batch_size* specify the number of training epochs and the minibatch size, respectively. Training uses the cross-entropy loss function and the [Adam optimizer](https://pytorch.org/docs/stable/optim.html#torch.optim.Adam) with learning rate *lr*. After each epoch, prints the perplexity of the model on the validation data.

The code in the cell below trains a bigram model.

In [12]:
model_fixed_window = train_fixed_window(2)

369.3
346.2


#### Performance goal

**Your submitted notebook must contain output demonstrating a validation perplexity of at most 350.** If you do not reach this perplexity after the first epoch, try training for a second epoch.

⚠️ Computing the validation perplexity in one go (for the full validation set) will probably exhaust your computer’s memory and/or take a lot of time. If you run into this problem, do the computation at the minibatch level and aggregate the results.

#### 🤞 Test your code

To see whether your network is learning something, print or plot the loss and/or the perplexity on the training data. If the two values do not decrease during training, try to find the problem before wasting time (and electricity) on useless computation.

Training and even evaluation will take some time – on a CPU, you should expect several minutes per epoch, depending on hardware. Our reference implementation uses a GPU and runs in less than 30 seconds per epoch on [Colab](http://colab.research.google.com).

## Problem 2: Recurrent neural network model

In this section, you will implement the recurrent neural network language model. Recall that an input to this model is a vector of word ids. Each integer is mapped to an embedding vector. The sequence of embedded vectors is then fed into an unrolled LSTM. At each position $i$ in the sequence, the hidden state of the LSTM at that position is sent through a linear transformation into a final softmax layer representing the probability distribution over the words at position $i+1$. In theory, the input vector could represent the complete training data; for practical reasons, however, we will truncate the input to some fixed value *bptt_len*. This length is called the **backpropagation-through-time horizon**.

### Problem 2.1: Vectorise the data (1&nbsp;point)

As in the previous problem, your first task is to transform the data in the WikiText container into a vectorised form that can be fed to the model.

In [13]:
class RNNVectorizer(object):
    def __init__(self, bptt_len):
        # backpropagation-through-time horizon
        self.bptt_len = bptt_len

    def __call__(self, data):
        all_sequences = []

        for i in range(0, len(data) - self.bptt_len, self.bptt_len):
          all_sequences.append((data[i:i+self.bptt_len + 1]))

        xs = []
        ys = []

        for sequence in all_sequences:
            xs.append(sequence[:-1])
            ys.append(sequence[1:])


        # Convert inputs and outputs into tensors
        tensor_x = torch.LongTensor(xs)
        tensor_y = torch.LongTensor(ys)

        return tensor_x, tensor_y

Your vectoriser should meet the following specification:

**__init__** (*self*, *bptt_len*)

> Creates a new vectoriser. The parameter *bptt_len* specifies the backpropagation-through-time horizon.

**__call__** (*self*, *data*)

> Transforms a list of token indexes *data* into a pair of tensors $\mathbf{X}$, $\mathbf{Y}$ that can be used to train the recurrent neural language model. The rows of both tensors represent contiguous subsequences of token indexes of length *bptt_len*. Compared to the sequences in $\mathbf{X}$, the corresponding sequences in $\mathbf{Y}$ are shifted one position to the right. More precisely, if the $i$th row of $\mathbf{X}$ is the sequence that starts at token position $j$, then the same row of $\mathbf{Y}$ is the sequence that starts at position $j+1$.

#### 🤞 Test your code

Test your implementation by running the following code:

In [14]:
valid_x, valid_y = RNNVectorizer(32)(wikitext.valid)

print(valid_x.size(), valid_y.size())

torch.Size([6837, 32]) torch.Size([6837, 32])


### Problem 2.2: Implement the model (2&nbsp;points)

Your next task is to implement the recurrent neural network model based on the graphical specification.

In [15]:
import torch.nn as nn

class RNNModel(nn.Module):

    def __init__(self, n_words, embedding_dim=50, hidden_dim=50):
        super().__init__()
        self.embedding = nn.Embedding(n_words, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first = True)
        self.linear = nn.Linear(hidden_dim, n_words)

    def forward(self, x):
        embedded_x = self.embedding(x)
        output_lstm, (h_n, c_n) = self.lstm(embedded_x)
        return self.linear(output_lstm)

Your implementation should follow this specification:

**__init__** (*self*, *n_words*, *embedding_dim* = 50, *hidden_dim* = 50)

> Creates a new recurrent neural network language model based on an LSTM. The argument *n_words* is the number of words in the vocabulary. The arguments *embedding_dim* and *hidden_dim* specify the dimensionalities of the embedding layer and the LSTM hidden layer, respectively; their default value is 50.

**forward** (*self*, *x*)

> Computes the network output on an input batch *x*. The shape of *x* is $(B, H)$, where $B$ is the batch size and $H$ is the length of each input sequence. The shape of the output tensor is $(B, H, V)$, where $V$ is the size of the vocabulary.

#### 🤞 Test your code

Test your code by instantiating the model and feeding it a batch of examples from the training data.

### Problem 2.3: Train the model (3&nbsp;points)

The training loop for the recurrent neural network model is essentially identical to the loop that you wrote for the feed-forward model. The only thing to note is that the cross-entropy loss function expects its input to be a two-dimensional tensor; you will therefore have to re-shape the output tensor from the LSTM as well as the gold-standard output tensor in a suitable way. The most efficient way to do so is to use the [`view()`](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.view) method.

In [16]:
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.nn.functional as F
import math

def train_rnn(n_epochs=2, batch_size=1000, bptt_len=32, lr=1e-2):


    # Initialise the model
    model = RNNModel(len(wikitext.vocab)).to(device)

    # Initialise the optimiser (stochastic gradient descent)
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # Initialise the data loader
    data_loader = DataLoader(wikitext.train, batch_size=batch_size, collate_fn=RNNVectorizer(bptt_len))
    valid_loader = DataLoader(wikitext.valid, batch_size=batch_size, collate_fn=RNNVectorizer(bptt_len))


    # We train for the specified number of epochs
    for epoch in range(n_epochs):

        total_loss = 0

        model.train()

        # In each epoch, we loop over the batches provided by the data loader
        for batch_x, batch_y in data_loader:

            # Reset the accumulated gradients
            optimizer.zero_grad()

            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)

            batch_y = batch_y.view(-1)

            # Forward pass
            output = model.forward(batch_x)

            output = output.view(-1, len(wikitext.vocab))

            # Compute the loss
            loss = F.cross_entropy(output, batch_y)

            # Backward pass: propagate the loss and compute the gradients
            loss.backward()

            # Update the parameters
            optimizer.step()


        running_loss = 0

        with torch.no_grad():

          model.eval()

          num_of_samples = 0

          # In each epoch, we loop over the batches provided by the data loader
          for batch_x, batch_y in valid_loader:

              batch_x = batch_x.to(device)

              batch_y = batch_y.to(device)

              batch_y = batch_y.view(-1)

              output = model.forward(batch_x)

              output = output.view(-1, len(wikitext.vocab))

              sample_size = len(batch_x)

              # Compute the loss
              loss = F.cross_entropy(output, batch_y) * sample_size
              num_of_samples += sample_size


              running_loss += loss

        # Compute the average
        total_loss = (running_loss / num_of_samples)

        # Print as perplexity
        print(f'{math.exp(total_loss):.1f}')


    return model

Here is the specification of the training function:

**train_rnn** (*n_epochs* = 1, *batch_size* = 1000, *bptt_len* = 32, *lr* = 0.01)

> Trains a recurrent neural network language model on the WikiText data using minibatch gradient descent and returns it. The parameters *n_epochs* and *batch_size* specify the number of training epochs and the minibatch size, respectively. The parameter *bptt_len* specifies the length of the backpropagation-through-time horizon, that is, the length of the input and output sequences. Training uses the cross-entropy loss function and the [Adam optimizer](https://pytorch.org/docs/stable/optim.html#torch.optim.Adam) with learning rate *lr*. After each epoch, prints the perplexity of the model on the validation data.

Evaluate your model by running the following code cell:

In [17]:
model_rnn = train_rnn()

301.1
269.6


#### Performance goal

**Your submitted notebook must contain output demonstrating a validation perplexity of at most 280.** If you do not reach this perplexity after the first epoch, try training for a second epoch.

## Problem 3: Parameter initialisation (3&nbsp;points)

The error surfaces explored when training neural networks can be very complex. Because of this, it is important to choose “good” initial values for the parameters. In PyTorch, the weights of the embedding layer are initialised by sampling from the standard normal distribution $\mathcal{N}(0, 1)$. Test how changing the initialisation affects the perplexity of your feed-forward language model. Find research articles that propose different initialisation strategies.

Write a short (150&nbsp;words) report about your experiments and literature search. Use the following prompts:

* What different initialisation did you try? What results did you get?
* How do your results compare to what was suggested by the research articles?
* What did you learn? How, exactly, did you learn it? Why does this learning matter?

You are allowed to consult sources for this problem if you appropriately cite them. If in doubt, please read the [Academic Integrity Policy](https://www.ida.liu.se/~TDDE09/logistics/policies.html#academic-integrity-policy).

In [20]:
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.nn.functional as F
import math

def train_fixed_windowXavier(n, n_epochs=2, batch_size=3200, lr=1e-2):


    # Initialise the model
    model = FixedWindowModel(n, len(wikitext.vocab)).to(device)

    nn.init.xavier_uniform_(model.e.weight.data)

    # Initialise the optimiser (stochastic gradient descent)
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # Initialise the data loader
    data_loader = DataLoader(wikitext.train, batch_size=batch_size, collate_fn=FixedWindowVectorizer(n))
    valid_loader = DataLoader(wikitext.valid, batch_size=batch_size, collate_fn=FixedWindowVectorizer(n))


    # We train for the specified number of epochs
    for epoch in range(n_epochs):

        total_loss = 0

        model.train()

        # In each epoch, we loop over the batches provided by the data loader
        for batch_x, batch_y in data_loader:

            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)

            # Reset the accumulated gradients
            optimizer.zero_grad()

            # Forward pass
            output = model.forward(batch_x)

            # Compute the loss
            loss = F.cross_entropy(output, batch_y)

            # Backward pass: propagate the loss and compute the gradients
            loss.backward()

            # Update the parameters
            optimizer.step()


        running_loss = 0

        with torch.no_grad():

          model.eval()
          num_of_samples = 0

          # In each epoch, we loop over the batches provided by the data loader
          for batch_x, batch_y in valid_loader:

              batch_x = batch_x.to(device)
              batch_y = batch_y.to(device)

              # Forward pass
              output = model.forward(batch_x)

              sample_size = len(batch_x)

              # Compute the loss
              loss = F.cross_entropy(output, batch_y) * sample_size
              num_of_samples += sample_size


              running_loss += loss

        # Compute the average
        total_loss = (running_loss / num_of_samples)

        # Print as perplexity
        print(f'{math.exp(total_loss):.1f}')


    return model

In [21]:
model_fixed_window = train_fixed_windowXavier(2)

340.8
317.6


In [22]:
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.nn.functional as F
import math

def train_fixed_windowHe(n, n_epochs=2, batch_size=3200, lr=1e-2):


    # Initialise the model
    model = FixedWindowModel(n, len(wikitext.vocab)).to(device)

    nn.init.kaiming_uniform_(model.e.weight.data)

    # Initialise the optimiser (stochastic gradient descent)
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # Initialise the data loader
    data_loader = DataLoader(wikitext.train, batch_size=batch_size, collate_fn=FixedWindowVectorizer(n))
    valid_loader = DataLoader(wikitext.valid, batch_size=batch_size, collate_fn=FixedWindowVectorizer(n))


    # We train for the specified number of epochs
    for epoch in range(n_epochs):

        total_loss = 0

        model.train()

        # In each epoch, we loop over the batches provided by the data loader
        for batch_x, batch_y in data_loader:

            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)

            # Reset the accumulated gradients
            optimizer.zero_grad()

            # Forward pass
            output = model.forward(batch_x)

            # Compute the loss
            loss = F.cross_entropy(output, batch_y)

            # Backward pass: propagate the loss and compute the gradients
            loss.backward()

            # Update the parameters
            optimizer.step()


        running_loss = 0

        with torch.no_grad():

          model.eval()
          num_of_samples = 0

          # In each epoch, we loop over the batches provided by the data loader
          for batch_x, batch_y in valid_loader:

              batch_x = batch_x.to(device)
              batch_y = batch_y.to(device)

              # Forward pass
              output = model.forward(batch_x)

              sample_size = len(batch_x)

              # Compute the loss
              loss = F.cross_entropy(output, batch_y) * sample_size
              num_of_samples += sample_size


              running_loss += loss

        # Compute the average
        total_loss = (running_loss / num_of_samples)

        # Print as perplexity
        print(f'{math.exp(total_loss):.1f}')


    return model

In [23]:
model_fixed_window = train_fixed_windowHe(2)

328.9
316.5


I tried the Xavier and He initialization, they were both much better than the standard normal deviation initialization. Xavier perplexity: 340.8 on first epoch and 317.6 on the second. He initialization: 328.9 on first and 316.5 on second. My results were close to the ones stated in the research article (https://openaccess.thecvf.com/content_iccv_2015/papers/He_Delving_Deep_into_ICCV_2015_paper.pdf), they said that Xavier doesnt work well together with the ReLu , but it worked better than the standard normal deviation and almost as well as the He, atleast during the second epoch, but during the first the He was much better. The difference between the initializations is that the Xavier only adress linearities (sigmoid activation function for example) while the He also adresses nonlinearities, which therefor should work better together with the ReLu. (https://openaccess.thecvf.com/content_iccv_2015/papers/He_Delving_Deep_into_ICCV_2015_paper.pdf). I also learned that the most important part of the weight initialization is the variance which both the He and Xavier utilises, they adapt the weigths to different activation functions so the input variance is the same as the output variance. (ChatGPT, prompt: "What is the difference between Xavier initialization and He initialization?")